In [48]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [49]:
RADIUS = 300

In [50]:
mrt_station = pd.read_csv('../MRT/mrt_station.csv')



In [51]:
# MRT_station_name: str
# city_code: str
# district: str
# station_lat: decimal
# station_lng: decimal

mrt_station = mrt_station[['StationName.Zh_tw','LocationCityCode', 'LocationTown','StationPosition.PositionLon',
       'StationPosition.PositionLat']]
mrt_station = mrt_station.rename(
    {
        'StationName.Zh_tw':'MRT_station_name',
        'LocationCityCode':'city_code',
        'LocationTown':'district',
        'StationPosition.PositionLat':'station_lat',
        'StationPosition.PositionLon':'station_lng'
    },axis=1)

mrt_station = mrt_station[['MRT_station_name','city_code','district','station_lat','station_lng']]

In [52]:
mrt_station = mrt_station.drop_duplicates(subset='MRT_station_name')

In [53]:
target_stations = mrt_station['MRT_station_name'].to_list()

In [54]:
# target_stations = ['民權西路','中山','台北車站','中正紀念堂',
#       '東門','大安','西門','忠孝新生','忠孝復興',
#       '南京復興','松江南京','古亭','西湖','港墘','南港']

In [55]:
mrt_station = mrt_station[mrt_station['city_code']=='TPE']

In [56]:
target_stations = mrt_station['MRT_station_name'].to_list()

In [57]:
bike_station = pd.read_csv('../bike/bike_usage_realtime.csv',index_col=0)
bike_station['sarea'] = bike_station['sarea'].replace('臺大公館校區','大安區')
bike_station = bike_station.rename(
    {
        'sna':'Bike_station_name',
        'sarea':'district',
        'latitude':'station_lat',
        'longitude':'station_lng'

    }
    ,axis=1)
bike_station['city_code']='TPE'

bike_station['Bike_station_name'] = bike_station['Bike_station_name'].apply(lambda x: x.split('_')[1])
bike_station = bike_station[['Bike_station_name','district',
                             'city_code','total','station_lat','station_lng']]

In [58]:
bike_coords = bike_station[['station_lat', 'station_lng']].to_numpy()
mrt_coords = mrt_station[['station_lat', 'station_lng']].to_numpy()

# Initialize a matrix to store distances
dist_matrix = np.zeros((len(mrt_coords), len(bike_coords)))

# Compute geodesic distance between each pair of points
for i in tqdm(range(len(mrt_coords))):
    for j in range(len(bike_coords)):
        dist = geopy.distance.geodesic(mrt_coords[i], bike_coords[j]).m
        dist_matrix[i, j] = dist

  0%|          | 0/73 [00:00<?, ?it/s]

100%|██████████| 73/73 [00:05<00:00, 13.65it/s]


In [59]:
ubike_rt = pd.read_csv('ubike_0501.csv',index_col=0)
ubike_rt.drop_duplicates(inplace=True)
ubike_rt['sarea'] = ubike_rt['sarea'].replace('臺大公館校區','大安區')
ubike_rt['sna'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])


In [60]:
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)
def almost_empty(row):
    bike = row['sbi']
    thres = row['thres']
    if bike <thres:
    # if bike <2:
        return True
    else:
        return False

In [61]:
ubike_rt['almost_empty'] = ubike_rt.apply(almost_empty,axis=1)
ubike_rt['is_empty'] = ubike_rt['sbi']==0
ubike_rt['date'] = pd.to_datetime(ubike_rt['updateTime']).dt.date
ubike_rt['hour'] = pd.to_datetime(ubike_rt['updateTime']).dt.hour
ubike_rt['min'] = (pd.to_datetime(ubike_rt['updateTime']).dt.minute)//10*10

In [62]:
ubike_rt = ubike_rt.drop(['ar', 'aren', 'infoDate', 'infoTime', 'mday','updateTime',
       'srcUpdateTime','sarea', 'sareaen', 'snaen', 'sno'],axis=1)

In [63]:
for idx, station in enumerate(target_stations):
    youbike_dist_to_station = dist_matrix[idx]
    youbike_index = np.where(youbike_dist_to_station<RADIUS)[0]
    print(station,bike_station.iloc[youbike_index].shape)
    # if idx==0:
    #     break

龍山寺 (5, 6)
西門 (7, 6)
台北車站 (4, 6)
善導寺 (7, 6)
忠孝新生 (9, 6)
忠孝復興 (7, 6)
忠孝敦化 (3, 6)
國父紀念館 (4, 6)
市政府 (5, 6)
永春 (3, 6)
後山埤 (7, 6)
昆陽 (5, 6)
南港 (7, 6)
南港展覽館 (6, 6)
動物園 (2, 6)
木柵 (1, 6)
萬芳社區 (1, 6)
萬芳醫院 (2, 6)
辛亥 (4, 6)
麟光 (3, 6)
六張犁 (4, 6)
科技大樓 (7, 6)
大安 (7, 6)
南京復興 (8, 6)
中山國中 (4, 6)
松山機場 (2, 6)
大直 (4, 6)
劍南路 (3, 6)
西湖 (7, 6)
港墘 (7, 6)
文德 (4, 6)
內湖 (4, 6)
大湖公園 (1, 6)
葫洲 (4, 6)
東湖 (3, 6)
南港軟體園區 (3, 6)
景美 (4, 6)
萬隆 (5, 6)
公館 (14, 6)
台電大樓 (9, 6)
古亭 (9, 6)
中正紀念堂 (4, 6)
小南門 (6, 6)
北門 (1, 6)
中山 (3, 6)
松江南京 (11, 6)
台北小巨蛋 (6, 6)
南京三民 (6, 6)
松山 (9, 6)
東門 (4, 6)
行天宮 (6, 6)
中山國小 (5, 6)
民權西路 (7, 6)
大橋頭 (7, 6)
象山 (4, 6)
台北101/世貿 (5, 6)
信義安和 (5, 6)
大安森林公園 (4, 6)
台大醫院 (2, 6)
雙連 (5, 6)
圓山 (4, 6)
劍潭 (7, 6)
士林 (4, 6)
芝山 (6, 6)
明德 (2, 6)
石牌 (4, 6)
唭哩岸 (3, 6)
奇岩 (3, 6)
北投 (4, 6)
復興崗 (2, 6)
忠義 (2, 6)
關渡 (4, 6)
新北投 (6, 6)


In [64]:
result = None
for idx, station in enumerate(target_stations):
    youbike_dist_to_station = dist_matrix[idx]
    youbike_index = np.where(youbike_dist_to_station<RADIUS)[0]
    bike_station_in_area = bike_station.iloc[youbike_index]
    subset = ubike_rt[ubike_rt['sna'].isin(bike_station_in_area['Bike_station_name'])]
    subset = subset.drop_duplicates(subset=['sna','date','hour','min'])
    subset = subset.reset_index(drop=True)
    subset = subset.groupby(['date','hour','min'],as_index=False)[['almost_empty','is_empty']].mean()
    single_mrt = mrt_station.iloc[idx].to_frame().T
    single_mrt['key'] = 0
    subset['key'] = 0
    single_mrt = single_mrt.merge(subset,on='key',how='outer')
    if result is None:
        result = single_mrt.copy(deep=True)
    else:
        result = pd.concat([result,single_mrt],ignore_index=True)
    

In [65]:
result

,MRT_station_name,city_code,district,station_lat,station_lng,key,date,hour,min,almost_empty,is_empty
0,龍山寺,TPE,萬華區,25.03528,121.500325,0,2024-04-16,22,20,0.500000,0.250000
1,龍山寺,TPE,萬華區,25.03528,121.500325,0,2024-04-16,22,30,0.500000,0.000000
2,龍山寺,TPE,萬華區,25.03528,121.500325,0,2024-04-16,22,40,0.500000,0.250000
3,龍山寺,TPE,萬華區,25.03528,121.500325,0,2024-04-16,22,50,0.500000,0.250000
4,龍山寺,TPE,萬華區,25.03528,121.500325,0,2024-04-16,23,0,0.500000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...
135483,新北投,TPE,北投區,25.136931,121.502595,0,2024-04-29,19,40,0.000000,0.000000
135484,新北投,TPE,北投區,25.136931,121.502595,0,2024-04-29,19,50,0.000000,0.000000
135485,新北投,TPE,北投區,25.136931,121.502595,0,2024-04-29,20,0,0.000000,0.000000
135486,新北投,TPE,北投區,25.136931,121.502595,0,2024-05-01,19,40,0.166667,0.166667


In [66]:
import datetime

In [67]:
def create_dt_from_row(row):
    year = row['date'].year
    month = row['date'].month
    day = row['date'].day
    return datetime.datetime(year,month,day,row['hour'],row['min'])

In [68]:
result['timestamp'] = result.apply(lambda row:create_dt_from_row(row),axis=1)

In [69]:
result.to_csv(f'pipeline_8_R{RADIUS}_all_station_dynamic_threshold.csv',index=False)